In [1]:
import torch
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler, random_split
import pandas as pd
import numpy as np
import os
import random
from collections import defaultdict, Counter
from itertools import combinations
import json
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import KFold
import optuna
from torchmetrics.classification import F1Score
import pickle

import sys
### import Dataset prepartion and model training classes from BS_LS_scripts folder
sys.path.insert(1, '/home/wangc90/circRNA/circRNA_Data/BS_LS_scripts/')
from BS_LS_DataSet_3 import BS_LS_DataSet_Prep, RCM_Score
from BS_LS_Training_Base_models_1 import Objective, Objective_CV


In [2]:
class RCM_optuna_flanking(nn.Module):
    '''
        This is for 2-d model to process the RCM score distribution of the flanking introns
    '''
    def __init__(self, trial):
        
        super(RCM_optuna_flanking, self).__init__()
        
        # convlayer 1
        self.out_channel1 = trial.suggest_categorical('flanking_out_channel1', [128, 256, 512])
#         self.out_channel1 = 128

#         kernel_size1 = 5

        self.conv1 = nn.Conv1d(in_channels=5, out_channels=self.out_channel1,\
                               kernel_size=5, stride=5, padding=0)
        self.conv1_bn = nn.BatchNorm1d(self.out_channel1)
        
        self.out_channel2 = trial.suggest_categorical('flanking_out_channel2', [128, 256, 512])
#         self.out_channel2 = 32
        
        self.conv2 = nn.Conv1d(in_channels=self.out_channel1, out_channels=self.out_channel2,\
                               kernel_size=5, stride=5, padding=0)
        
        self.conv2_bn = nn.BatchNorm1d(self.out_channel2)
        
        self.conv2_out_dim = 10
    

    def forward(self, x):
        out = x
        out = torch.relu(self.conv1_bn(self.conv1(out)))
        out = torch.relu(self.conv2_bn(self.conv2(out)))

        out = out.view(-1, self.out_channel2 * self.conv2_out_dim)
        return out

    
class RCM_optuna_upper(nn.Module):
    '''
        This is for 2-d model to process the RCM score distribution of the upper introns
    '''
    def __init__(self, trial):
        
        super(RCM_optuna_upper, self).__init__()
        
        # convlayer 1
        self.out_channel1 = trial.suggest_categorical('upper_out_channel1', [128, 256, 512])
#         self.out_channel1 = 512

        self.conv1 = nn.Conv1d(in_channels=5, out_channels=self.out_channel1,\
                               kernel_size=5, stride=5, padding=0)
        
        self.conv1_bn = nn.BatchNorm1d(self.out_channel1)
        
        self.out_channel2 = trial.suggest_categorical('upper_out_channel2', [128, 256, 512])
#         self.out_channel2 = 64
        
        self.conv2 = nn.Conv1d(in_channels=self.out_channel1, out_channels=self.out_channel2,\
                               kernel_size=5, stride=5, padding=0)
        
        self.conv2_bn = nn.BatchNorm1d(self.out_channel2)
        self.conv2_out_dim = 10
    

    def forward(self, x):
        out = x
        out = torch.relu(self.conv1_bn(self.conv1(out)))
        out = torch.relu(self.conv2_bn(self.conv2(out)))

        out = out.view(-1, self.out_channel2 * self.conv2_out_dim)
        return out
    
    
class RCM_optuna_lower(nn.Module):
    '''
        This is for 2-d model to process the RCM score distribution of the lower introns
    '''
    def __init__(self, trial):
        
        super(RCM_optuna_lower, self).__init__()
        
        # convlayer 1
        self.out_channel1 = trial.suggest_categorical('lower_out_channel1', [128, 256, 512])
#         self.out_channel1 = 512

        self.conv1 = nn.Conv1d(in_channels=5, out_channels=self.out_channel1,\
                               kernel_size=5, stride=5, padding=0)
        
        self.conv1_bn = nn.BatchNorm1d(self.out_channel1)
        self.out_channel2 = trial.suggest_categorical('lower_out_channel2', [128, 256, 512])
#         self.out_channel2 = 512
        
        self.conv2 = nn.Conv1d(in_channels=self.out_channel1, out_channels=self.out_channel2,\
                               kernel_size=5, stride=5, padding=0)
        
        self.conv2_bn = nn.BatchNorm1d(self.out_channel2)
        self.conv2_out_dim = 10
    

    def forward(self, x):
        out = x
        out = torch.relu(self.conv1_bn(self.conv1(out)))
        out = torch.relu(self.conv2_bn(self.conv2(out)))

        out = out.view(-1, self.out_channel2 * self.conv2_out_dim)
        return out
    
    

class RCM_optuna_concate(nn.Module):
    ''''
        
    '''
    def __init__(self, trial):
        
        super(RCM_optuna_concate, self).__init__()

        ### cnn for the flanking rcm scores
        self.cnn_flanking = RCM_optuna_flanking(trial)

        self.flanking_out_dim = self.cnn_flanking.conv2_out_dim
        self.flanking_out_channel = self.cnn_flanking.out_channel2
#         print(f'flanking out dim: {self.flanking_out_dim}, flanking out channel {self.flanking_out_channel}')
        
        ### cnn for the upper rcm scores
        self.cnn_upper = RCM_optuna_upper(trial)

        self.upper_out_dim = self.cnn_upper.conv2_out_dim
        self.upper_out_channel = self.cnn_upper.out_channel2
#         print(f'upper_out_dim: {self.upper_out_dim}, upper_out_channel {self.upper_out_channel}')
        
        ### cnn for the lower rcm scores
        self.cnn_lower = RCM_optuna_lower(trial)

        self.lower_out_dim = self.cnn_lower.conv2_out_dim
        self.lower_out_channel = self.cnn_lower.out_channel2
#         print(f'lower_out_dim: {self.lower_out_dim}, lower_out_channel {self.lower_out_channel}')
        

        self.fc1_input_dim = self.flanking_out_dim * self.flanking_out_channel + \
                             self.upper_out_dim * self.upper_out_channel + \
                             self.lower_out_dim * self.lower_out_channel

#         print(f'fc1_input_dim: {self.fc1_input_dim}')
        
        
        self.fc1_out = trial.suggest_categorical('concat_fc1_out', [128, 256, 512])
#         self.fc1_out = 512
    
        # add the rcm feature dimension here as well (5*5+2)*3+2 = 83
        self.fc1 = nn.Linear(self.fc1_input_dim, self.fc1_out)
        
        self.fc1_bn = nn.BatchNorm1d(self.fc1_out)

        dropout_rate_fc1 = trial.suggest_categorical("concat_dropout_rate_fc1",  [0, 0.1, 0.2, 0.4])
        self.drop_nn1 = nn.Dropout(p=dropout_rate_fc1)

        # fc layer2
        # use dimension output with nn.CrossEntropyLoss()
        self.fc2_out = trial.suggest_categorical('concat_fc2_out', [4, 8, 16, 32])
#         self.fc2_out = 8
        self.fc2 = nn.Linear(self.fc1_out, self.fc2_out)

        self.fc2_bn = nn.BatchNorm1d(self.fc2_out)

        dropout_rate_fc2 = trial.suggest_categorical("concat_dropout_rate_fc2",[0, 0.1, 0.2, 0.4])
    
        self.drop_nn2 = nn.Dropout(p=dropout_rate_fc2)

        self.fc3 = nn.Linear(self.fc2_out, 2)
        

    def forward(self, rcm_flanking, rcm_upper, rcm_lower):
        
        x1 = self.cnn_flanking(rcm_flanking)

        x2 = self.cnn_upper(rcm_upper)
        
        x3 = self.cnn_lower(rcm_lower)
        
        x = torch.cat((x1,x2,x3), dim=1)
    
        # feed the concatenated feature to fc1
        out = self.fc1(x)
        out = self.drop_nn1(torch.relu(self.fc1_bn(out)))
        out = self.fc2(out)
        out = self.drop_nn2(torch.relu(self.fc2_bn(out)))
        out = self.fc3(out)
        return out


In [3]:
def rcm_flankingWithin_all_windows_optuna(num_trial):
    
    ## specify different kmer length to get the training data of the rcm score for that kmer 
    ### just change this number to 10, 20, 40 and 80 to get the model performance for different kmer length

    study = optuna.create_study(direction='maximize')

    ### where to save the 3-fold CV validation acc based on the rcm score and mlp

    val_acc_folder = f'/home/wangc90/circRNA/circRNA_Data/model_outputs/rcm_flankingWithin_all_windows/9000/val_acc_cv3'
    ### where to save the best model in the 3-fold CV 
    ### wehre to save the detailed optuna results
    optuna_folder = f'/home/wangc90/circRNA/circRNA_Data/model_outputs/rcm_flankingWithin_all_windows/9000/optuna'
    
    
    BS_LS_coordinates_path = '/home/wangc90/circRNA/circRNA_Data/BS_LS_data/updated_data/BS_LS_coordinates_final.csv'
    hg19_seq_dict_json_path = '/home/wangc90/circRNA/circRNA_Data/hg19_seq/hg19_seq_dict.json'
    flanking_dict_folder = '/home/wangc90/circRNA/circRNA_Data/BS_LS_data/flanking_dicts/'
    bs_ls_dataset = BS_LS_DataSet_Prep(BS_LS_coordinates_path=BS_LS_coordinates_path,
                                   hg19_seq_dict_json_path=hg19_seq_dict_json_path,
                                   flanking_dict_folder=flanking_dict_folder,
                                   flanking_junction_bps=100,
                                   flanking_intron_bps=5000,
                                   training_size=9000)


    ## generate the junction and flanking intron dict
    bs_ls_dataset.get_junction_flanking_intron_seq()

    train_key_1, _, test_keys = bs_ls_dataset.get_train_test_keys()

    rcm_scores_folder = '/home/wangc90/circRNA/circRNA_Data/BS_LS_data/flanking_dicts/rcm_scores/'

    ### try with rcm features
    _, _, train_torch_flanking_rcm, train_torch_upper_rcm, train_torch_lower_rcm, \
        train_torch_labels = bs_ls_dataset.seq_to_tensor(data_keys=train_key_1,\
                                                         rcm_folder=rcm_scores_folder,\
                                                         is_rcm=True, is_upper_lower_concat=False)
    
#     print(train_torch_flanking_rcm.shape)

    RCM_kmer_Score_dataset = RCM_Score(flanking_only=False,
                                       flanking_rcm=train_torch_flanking_rcm,\
                                       upper_rcm=train_torch_upper_rcm,\
                                       lower_rcm=train_torch_lower_rcm,\
                                       label=train_torch_labels)
    print(len(RCM_kmer_Score_dataset))
    
#     RCM_kmer_Score_dataset = RCM_Score(rcm=train_torch_rcm, label=train_torch_labels)

    study = optuna.create_study(pruner=optuna.pruners.MedianPruner(n_warmup_steps=1, n_startup_trials=10),
                                direction='maximize')

    study.optimize(Objective_CV(cv=3, model= RCM_optuna_concate, 
                                dataset=RCM_kmer_Score_dataset,
                                val_acc_folder=val_acc_folder), n_trials=num_trial, gc_after_trial=True)


    pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
    complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]
    with open(optuna_folder+'/optuna.txt', 'a') as f:
        f.write("Study statistiqcs: \n")
        f.write(f"Number of finished trials: {len(study.trials)}\n")
        f.write(f"Number of pruned trials: {len(pruned_trials)}\n")
        f.write(f"Number of complete trials: {len(complete_trials)}\n")

        f.write("Best trial:\n")
        trial = study.best_trial
        f.write(f"Value: {trial.value}\n")
        f.write("Params:\n")
        for key, value in trial.params.items():
            f.write(f"{key}:{value}\n")

    df = study.trials_dataframe().drop(['state','datetime_start','datetime_complete','duration','number'], axis=1)
    df.to_csv(optuna_folder + '/optuna.csv', sep='\t', index=None)

In [ ]:
rcm_flankingWithin_all_windows_optuna(num_trial=500)

[I 2023-11-22 19:17:52,285] A new study created in memory with name: no-name-537499c0-6fd3-4536-a7ba-15fbd7003d4a


chr5|138837130|138837392|- has N in the extracted junctions, belongs to BS
There are 0 overlapped flanking sequence from BS and LS  
There are 7 repeated BS sequences
There are 2 repeated LS sequences


[I 2023-11-22 19:20:06,239] A new study created in memory with name: no-name-261e7957-6a39-457e-abfc-387db6819c9f


18000
fold 1, epoch 20, val loss 8.109893918037415 val accuracy 0.5852
fold 2, epoch 20, val loss 8.514750123023987 val accuracy 0.5542
fold 3, epoch 20, val loss 8.191138446331024 val accuracy 0.5663


[I 2023-11-22 19:27:15,868] Trial 0 finished with value: 0.5738666666666666 and parameters: {'lr': 1.0821414379716625e-05, 'l2_lambda': 1.1685422985912582e-07, 'batch_size': 512, 'epochs': 30, 'flanking_out_channel1': 256, 'flanking_out_channel2': 256, 'upper_out_channel1': 256, 'upper_out_channel2': 256, 'lower_out_channel1': 256, 'lower_out_channel2': 256, 'concat_fc1_out': 128, 'concat_dropout_rate_fc1': 0.1, 'concat_fc2_out': 8, 'concat_dropout_rate_fc2': 0.4}. Best is trial 0 with value: 0.5738666666666666.


fold 1, epoch 20, val loss 234.15906316041946 val accuracy 0.6182
fold 1, epoch 40, val loss 309.19565188884735 val accuracy 0.6162
fold 1, epoch 60, val loss 368.58680564165115 val accuracy 0.6072
fold 2, epoch 20, val loss 233.26067835092545 val accuracy 0.613
fold 2, epoch 40, val loss 330.061268389225 val accuracy 0.6128
fold 2, epoch 60, val loss 350.02090483903885 val accuracy 0.619
fold 3, epoch 20, val loss 236.32718884944916 val accuracy 0.6077
fold 3, epoch 40, val loss 321.0783626139164 val accuracy 0.6093


[I 2023-11-22 20:06:17,182] Trial 1 finished with value: 0.6134666666666666 and parameters: {'lr': 0.00011758819234934801, 'l2_lambda': 4.9891074956110835e-09, 'batch_size': 32, 'epochs': 60, 'flanking_out_channel1': 256, 'flanking_out_channel2': 128, 'upper_out_channel1': 512, 'upper_out_channel2': 256, 'lower_out_channel1': 512, 'lower_out_channel2': 256, 'concat_fc1_out': 128, 'concat_dropout_rate_fc1': 0, 'concat_fc2_out': 8, 'concat_dropout_rate_fc2': 0.4}. Best is trial 1 with value: 0.6134666666666666.


fold 3, epoch 60, val loss 382.0327248573303 val accuracy 0.6142
fold 1, epoch 20, val loss 320.6615119576454 val accuracy 0.6228
fold 1, epoch 40, val loss 351.4299221634865 val accuracy 0.6325
fold 1, epoch 60, val loss 392.577030479908 val accuracy 0.6328
fold 1, epoch 80, val loss 414.0701285004616 val accuracy 0.6258
fold 2, epoch 20, val loss 296.36080038547516 val accuracy 0.6212
fold 2, epoch 40, val loss 351.4863486289978 val accuracy 0.629
fold 2, epoch 60, val loss 366.1634866297245 val accuracy 0.6368
fold 2, epoch 80, val loss 394.08050557971 val accuracy 0.6368
fold 3, epoch 20, val loss 305.4903134703636 val accuracy 0.6147
fold 3, epoch 40, val loss 356.15211966633797 val accuracy 0.6303
fold 3, epoch 60, val loss 376.6057547926903 val accuracy 0.6313
fold 3, epoch 80, val loss 413.0440573692322 val accuracy 0.625


[I 2023-11-22 21:14:35,992] Trial 2 finished with value: 0.6321666666666667 and parameters: {'lr': 0.00045321404029266164, 'l2_lambda': 3.746971953372159e-07, 'batch_size': 32, 'epochs': 90, 'flanking_out_channel1': 512, 'flanking_out_channel2': 256, 'upper_out_channel1': 256, 'upper_out_channel2': 512, 'lower_out_channel1': 512, 'lower_out_channel2': 512, 'concat_fc1_out': 512, 'concat_dropout_rate_fc1': 0, 'concat_fc2_out': 32, 'concat_dropout_rate_fc2': 0.4}. Best is trial 2 with value: 0.6321666666666667.


fold 1, epoch 20, val loss 60.64893889427185 val accuracy 0.6312
fold 1, epoch 40, val loss 60.98318600654602 val accuracy 0.6427
fold 1, epoch 60, val loss 62.26195728778839 val accuracy 0.6433
fold 1, epoch 80, val loss 64.40980935096741 val accuracy 0.6473
fold 1, epoch 100, val loss 68.16363334655762 val accuracy 0.6475
fold 1, epoch 120, val loss 71.35802870988846 val accuracy 0.65
fold 2, epoch 20, val loss 63.241135120391846 val accuracy 0.6232
fold 2, epoch 40, val loss 62.04820740222931 val accuracy 0.6395
fold 2, epoch 60, val loss 62.08351355791092 val accuracy 0.6493
fold 2, epoch 80, val loss 64.14801225066185 val accuracy 0.6473
fold 2, epoch 100, val loss 67.56939125061035 val accuracy 0.6517
fold 2, epoch 120, val loss 69.49634838104248 val accuracy 0.6498
fold 3, epoch 20, val loss 61.390293419361115 val accuracy 0.623
fold 3, epoch 40, val loss 60.49144262075424 val accuracy 0.6388
fold 3, epoch 60, val loss 60.6407128572464 val accuracy 0.6453
fold 3, epoch 80, val l